### Recap from previous Lab

* We explored PyTorch (PT) tensor support and drew parallelisms w.r.t. NumPy `ndarray`
* We experimented with some basic Machine Learning (ML) building blocks in PT
* We built our first, very simple Artificial Neural Network (ANN), a MultiLayer Perceptrion (MLP)

### Agenda for today

* Today we will be training a MLP using SGD and backpropagation
  * we will not be using the synthetic dataset we saw last lecture
  * we will be training our MLP on the MNIST dataset, which is a simple (in modern terms) dataset on which to train our ANNs

### Intro to MNIST

MNIST is a dataset for **handwritten digit recognition**.

![](img/mnist.png)

* The dataset is composed of 60,000 grayscale images
  * by default, the dataset is already split into a training set of 50,000 images, while the remaining 10,000 images make up the test/validation set
* Each image is composed of 28x28 pixel
* Only one digit is present in each image
  * thus, we will be classifying digits from 0 to 9 (10 classes)
* The digit is centered within the image

#### Downloading the data

Since we're not covering the handling of data in this specific tutorial, I have prepared an external script which will download the data and "pack" it into DataLoaders.
You just need to know that DataLoaders and Datasets are two different entities; namely, DataLoaders are build on top of Datasets and handle the creation of the mini-batches that will later be fed into the MLP for the training and testing phase.

The script returns both the DataLoaders and Datasets, in case you wanna play with them

In [1]:
import torch
from scripts import mnist
from matplotlib import pyplot as plt

minibatch_size_train = 256
minibatch_size_test = 512

trainloader, testloader, trainset, testset = mnist.get_data(batch_size_train=minibatch_size_test, batch_size_test=minibatch_size_test)

ImportError: numpy.core.multiarray failed to import

In [2]:
plt.imshow(trainset.data[15].numpy(), cmap="gray")
